In [14]:
import json
from functools import wraps
from typing import Any, Callable
from pydantic import validate_arguments, BaseModel
import openai
from dotenv import load_dotenv
import os
import io
import contextlib

In [12]:
load_dotenv(".env")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def _remove_a_key(d, remove_key) -> None:
    """Remove a key from a dictionary recursively"""
    if isinstance(d, dict):
        for key in list(d.keys()):
            if key == remove_key:
                del d[key]
            else:
                _remove_a_key(d[key], remove_key)

In [4]:
class openai_function:
    def __init__(self, func: Callable) -> None:
        self.func = func
        self.validate_func = validate_arguments(func)
        parameters = self.validate_func.model.schema()
        parameters["properties"] = {
            k: v
            for k, v in parameters["properties"].items()
            if k not in ("v__duplicate_kwargs", "args", "kwargs")
        }
        parameters["required"] = sorted(
            parameters["properties"]
        )  # bug workaround see lc
        _remove_a_key(parameters, "title")
        _remove_a_key(parameters, "additionalProperties")
        self.openai_schema = {
            "name": self.func.__name__,
            "description": self.func.__doc__,
            "parameters": parameters,
        }
        self.model = self.validate_func.model

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        @wraps(self.func)
        def wrapper(*args, **kwargs):
            return self.validate_func(*args, **kwargs)

        return wrapper(*args, **kwargs)

    def from_response(self, completion, throw_error=True):
        """Execute the function from the response of an openai chat completion"""
        message = completion.choices[0].message

        if throw_error:
            assert "function_call" in message, "No function call detected"
            assert (
                message["function_call"]["name"] == self.openai_schema["name"]
            ), "Function name does not match"

        function_call = message["function_call"]
        arguments = json.loads(function_call["arguments"])
        return self.validate_func(**arguments)


class OpenAISchema(BaseModel):
    @classmethod
    @property
    def openai_schema(cls):
        schema = cls.schema()
        parameters = {
            k: v for k, v in schema.items() if k not in ("title", "description")
        }
        parameters["required"] = sorted(parameters["properties"])
        _remove_a_key(parameters, "title")
        return {
            "name": schema["title"],
            "description": schema["description"],
            "parameters": parameters,
        }

    @classmethod
    def from_response(cls, completion, throw_error=True):
        message = completion.choices[0].message

        if throw_error:
            assert "function_call" in message, "No function call detected"
            assert (
                message["function_call"]["name"] == cls.openai_schema["name"]
            ), "Function name does not match"

        function_call = message["function_call"]
        arguments = json.loads(function_call["arguments"])
        return cls(**arguments)

In [5]:
@openai_function
def sum(a:int, b:int) -> int:
    """Sum description adds a + b"""
    return a + b

In [18]:
@openai_function
def execute_code(code):
    """execute_code description executes python code"""
    buffer = io.StringIO()
    with contextlib.redirect_stdout(buffer):
        try:
            exec(code, globals())
        except Exception as e:
            print(f"Error: {e}")
    output = buffer.getvalue()
    buffer.close()
    return output

In [59]:
completion = openai.ChatCompletion.create(
        model="gpt-4-0613",
        temperature=0,
        functions=[execute_code.openai_schema],
        messages=[
            {
                "role": "system",
                "content": "You must use the `execute_code` function instead of writing python code. Make sure to always include a print() statement and print your answers",
            },
            {
                "role": "user",
                "content": "build a diagram for a convolutional neural net using python. Instead of writing Python, use the `execute_code` function. Make sure to include a print() statment for your answer",
            },
        ],
    )

print(completion)

{
  "id": "chatcmpl-7UQ4X4Bl8uBmmkK7OClxgTvBPQnqu",
  "object": "chat.completion",
  "created": 1687484417,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I'm sorry for the confusion, but it's not possible to build a diagram using the `execute_code` function. The `execute_code` function is used to execute Python code, not to create diagrams. However, you can use Python libraries such as matplotlib or seaborn to create diagrams, but these diagrams can't be displayed using the `execute_code` function. \n\nIf you want to build a diagram for a convolutional neural network, you can use libraries such as Keras or PyTorch, which provide functions to visualize the architecture of neural networks. However, these diagrams are typically displayed in a Jupyter notebook or other interactive Python environments, not through a function like `execute_code`. \n\nIf you're working in a Jupyter notebook, here's an exampl

In [60]:
result = execute_code.from_response(completion)
print(result)

AssertionError: No function call detected